In [ ]:
# ✅ File: notebooks/02_ab_testing.ipynb (Python script version for conversion)

# This simulates sending multiple requests to both variants of the deployed SageMaker endpoint

import boto3
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load TF-IDF vectorizer
vectorizer = joblib.load("../data/processed/tfidf_vectorizer.pkl")

runtime = boto3.client("sagemaker-runtime")
endpoint_name = "sms-ab-endpoint"

# Test messages
samples = [
    "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123 to claim now.",
    "Hey, are we still meeting for lunch today?",
    "URGENT! Your account has been compromised. Respond immediately!",
    "I'll call you when I get off work."
]

# Vectorize
X = vectorizer.transform(samples).toarray()

# Invoke endpoint for each sample
for i, x in enumerate(X):
    payload = json.dumps(x.tolist())
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=payload
    )
    prediction = json.loads(response["Body"].read().decode())
    print(f"Sample {i+1}: Prediction: {prediction}")
